In [14]:
import opendatasets as od
import numpy as np
import re
import pandas as pd

In [13]:
od.download('https://www.kaggle.com/datasets/atmarouane/en-partofspeech-tags?select=pos_tags.csv')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: vasanth12
Your Kaggle Key: ········


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22.0M/22.0M [00:08<00:00, 2.65MB/s]


In [ ]:
data = pd.read_csv('./en-partofspeech-tags/')

In [3]:
removeNonWords = re.compile(r'\W')

In [4]:
states = {}
observables = {}

In [5]:
countStates = 1
countObservables = 0

for taggedSentence in brown.tagged_sents(tagset='universal'):
    for token in taggedSentence:
        word = token[0]
        if removeNonWords.match(word):
            continue
        if states.get(token[1]) == None:
            states[token[1]] = countStates
            countStates += 1
        if observables.get(word) == None:
            observables[word] = countObservables
            countObservables += 1
    
print("countObservables: ", countObservables)
print("countStates: ", countStates)
#print("states: ",states)
#print("observables: ",observables)
    

countObservables:  55640
countStates:  12


In [6]:
removeNonWords.match("there's,")

In [7]:
transitionMatrix = np.zeros((countStates, countStates))
emissionMatrix = np.zeros((countStates, countObservables))

In [8]:
for taggedSentence in brown.tagged_sents(tagset='universal'):
    previousState = 0
    for token in taggedSentence:
        word = token[0]
        if removeNonWords.match(word):
            continue
        wordIndex = observables[word]
        stateIndex = states[token[1]]
        
        transitionMatrix[previousState][stateIndex] += 1
        emissionMatrix[stateIndex][wordIndex] += 1
        
        previousState = stateIndex
        

In [9]:
for tag, index in states.items():
    print(tag, "->",index,end="\t")

DET -> 1	NOUN -> 2	ADJ -> 3	VERB -> 4	ADP -> 5	ADV -> 6	CONJ -> 7	PRT -> 8	PRON -> 9	NUM -> 10	X -> 11	

In [10]:
for i in range(countStates):
    for j in range(countStates):
        print(emissionMatrix[i][j], end=",\t")
    print()

0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	
7258.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	3529.0,	0.0,	0.0,	0.0,	0.0,	
0.0,	17.0,	85.0,	0.0,	4.0,	0.0,	60.0,	0.0,	43.0,	0.0,	4.0,	0.0,	
0.0,	0.0,	0.0,	15.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	167.0,	
0.0,	0.0,	0.0,	0.0,	0.0,	1943.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	
0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	36078.0,	0.0,	0.0,	
0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	
0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	11.0,	0.0,	0.0,	0.0,	0.0,	
0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	
0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	
0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	
0.0,	0.0,	0.0,	3.0,	0.0,	0.0,	0.0,	2.0,	0.0,	2.0,	0.0,	0.0,	


In [14]:
states.keys()

dict_keys(['DET', 'NOUN', 'ADJ', 'VERB', 'ADP', 'ADV', 'CONJ', 'PRT', 'PRON', 'NUM', 'X'])

# simple example of Viterbi Algorithm

import numpy as np

obs = {
    "normal": 0, 
    "cold": 1, 
    "dizzy": 2
}

states = {
    "Healthy": 0,
    "Fever": 1
}

start_p = [0.6, 0.4]

trans_p = np.array([[0.7, 0.3], [0.4, 0.6]])

emit_p = np.array([[0.5, 0.4, 0.1], [0.1, 0.3, 0.6]])

inputObs = ['normal', 'normal', 'dizzy']

res = viterbi(states, obs, start_p, trans_p, emit_p, inputObs)

print(res)
##### [0, 0, 1,]


In [29]:
def viterbi(states, observables, stationaryTrans, transitionProbability, emissionProbability, inputSent):
    
    N = len(states.keys())
    M = len(inputSent)
    
    t1 = np.zeros((N,M+1))
    t2 = np.zeros((N,M+1))
    
    #y = lambda word:observables.get(word) in inputSent
    y = []
    for obs in inputSent:
        y.append(observables[obs])
    
    for i in range(N):
        t1[i][0] = stationaryTrans[i] * emissionProbability[i][y[0]]

    x = -1
    for j in range(1, M+1):
        for i in range(N):
            _max = -1000
            argmax = -1
            for k in range(N):
                try:
                    abc = t1[k][j-1] * transitionProbability[k][i] * emissionProbability[i][y[j]]
                    if _max < abc:
                        _max = abc
                        argmax = k
                except:
                    abc = t1[k][j-1]
                    if _max < abc:
                        _max = abc
                        argmax = k
            t1[i][j] = _max 
            t2[i][j] = argmax
            x = argmax
            
    res = np.zeros(M)
    for i in range(M, 0, -1):
        res[i-1] = t2[x][i]
        x = int(res[i-1])
        
    return res
        